## Missing Fingers

In [1]:
import warnings
from typing import List

import pyro
import pyro.distributions as dist
import torch

import probability_on_trial as pot

Building Core model:

In [2]:
pyro.set_rng_seed(42)



pr_identity = 0.01
rmp_fingers_sweden = 2 / 20000
pr_isis_if_not_guilty = 300 / 10.6e6
pr_fingers_match_if_isis=2 / 5000


def swedish_fingers_guilt_model(
    pr_identity: float = pr_identity,  # Prior probability that the suspect is guilty
    rmp_fingers_sweden=rmp_fingers_sweden,  # RMP for Sweden
    pr_isis_if_not_guilty= pr_isis_if_not_guilty, # around 300 ISI fighters from Sweden
    pr_fingers_match_if_isis= pr_fingers_match_if_isis,  # RMP for ISIS memebers; for now a precise number
    num_particles=3e6,
):

    with pyro.plate("particles", size=num_particles):

        guilty = pyro.sample("guilty", dist.Bernoulli(pr_identity))

        isis = pyro.sample(
            "isis", dist.Bernoulli(guilty * 1 + (1 - guilty) * pr_isis_if_not_guilty)
        )

        fingers_match_sweden = pyro.sample(
            "fingers_match_not_isis",
            # if guilty then 1
            # if not guilty then sample with probability rmp
            dist.Bernoulli(guilty * 1 + (1 - guilty) * (rmp_fingers_sweden)),
        )


        # this is with isis as an informative node, but without conditionin on it
        # conditioning will be achieved by masking
        fingers_match_isis = pyro.sample(
            "fingers_match_isis",
            # if guilty then 1
            # if not guilty then sample with probability pr_fingers_match_if_isis
            dist.Bernoulli(
                guilty * 1
                + (1 - guilty)
                * (
                    (pr_fingers_match_if_isis) * isis
                    + (1 - isis) * (rmp_fingers_sweden)
                )
            ),
        )

    # Mask 1: fingers match
    evidence_mask_fingers_sweden = (fingers_match_sweden).bool()
    guilty_masked_fingers_sweden = pyro.deterministic(
        "guilty_masked_fingers_sweden",
        torch.where(evidence_mask_fingers_sweden, guilty, torch.tensor(float("nan"))),
    )

    posterior_guilty_fingers_sweden = pyro.deterministic(
        "posterior_guilty_fingers_sweden", torch.nanmean(guilty_masked_fingers_sweden, axis=0)
    )

    # Mask 2: fingers match, isis
    evidence_mask_fingers_isis = (fingers_match_isis * isis).bool()
    guilty_masked_fingers_isis = pyro.deterministic(
        "guilty_masked_fingers_isis",
        torch.where(evidence_mask_fingers_isis, guilty, torch.tensor(float("nan"))),
    )

    posterior_guilty_fingers_isis = pyro.deterministic(
        "posterior_guilty_fingers_isis"        ,
        torch.nanmean(guilty_masked_fingers_isis, axis=0),
    )

    return {
        "guilty": guilty,
        "isis": isis,
        "fingers_match_sweden": fingers_match_sweden,
        "fingers_match_isis": fingers_match_isis,
        "posterior_guilty_fingers_sweden": posterior_guilty_fingers_sweden,
        "posterior_guilty_fingers_isis": posterior_guilty_fingers_isis,
    }


# set pyro random seed
pyro.set_rng_seed(422323)

swedish_fingers_guilt_model()

{'guilty': tensor([0., 0., 0.,  ..., 0., 0., 0.]),
 'isis': tensor([0., 0., 0.,  ..., 0., 0., 0.]),
 'fingers_match_sweden': tensor([0., 0., 0.,  ..., 0., 0., 0.]),
 'fingers_match_isis': tensor([0., 0., 0.,  ..., 0., 0., 0.]),
 'posterior_guilty_fingers_sweden': tensor(0.9895),
 'posterior_guilty_fingers_isis': tensor(1.)}

In [5]:
# requires pyro 1.9.1
#pyro.render_model(swedish_fingers_guilt_model, render_deterministic=True)

In [6]:
# extracting the posterior probabilities

with pyro.poutine.trace() as tr:
    pyro.set_rng_seed(42)
    swedish_fingers_guilt_model(num_particles=1e6)

for key in tr.trace.nodes.keys():
    if key.startswith("posterior"):
        print(key, tr.trace.nodes[key]["value"])

posterior_guilty_fingers_sweden tensor(0.9897)
posterior_guilty_fingers_isis tensor(1.)


In [5]:
posterior_guilt_sweden = torch.tensor(
    [
        1 - tr.trace.nodes["posterior_guilty_fingers_sweden"]["value"],
        tr.trace.nodes["posterior_guilty_fingers_sweden"]["value"],
    ]
)

posterior_guilt_isis = torch.tensor(
    [
        1 - tr.trace.nodes["posterior_guilty_fingers_isis"]["value"],
        tr.trace.nodes["posterior_guilty_fingers_isis"]["value"],
    ]
)

print("posterior_guilt_not_isis_mean", posterior_guilt_sweden)
print("posterior_guilt_isis_mean", posterior_guilt_isis)



prior_guilt = torch.tensor([1 - pr_identity, pr_identity])


probs_of_evidence = torch.tensor(
    [pr_fingers_match_if_not_isis, pr_fingers_match_if_isis]
)
print(probs_of_evidence)

KeyError: 'posterior_guilt_not_isis_mean'

In [54]:
expected_weights = expected_weight(
    probs_of_evidence,
    prior_guilt,
    [posterior_guilt_not_isis_mean, posterior_guilt_isis_mean],
)

print("Expected Weights:", expected_weights)

tensor([0.9179, 0.9160])
Expected Weights: {'expected_weight': tensor(-1.4350e-06), 'weight_prior': 0.9192068800330162, 'posterior_weights': tensor([0.9179, 0.9160]), 'weight_changes': tensor([-0.0013, -0.0033]), 'weighted_weight_changes': tensor([-1.3228e-07, -1.3027e-06])}
